<a align="left" href="https://www.fondationvallet.org/" target="_blank">
<img width="1024", src="https://www.fondationvallet.org/wp-content/uploads/2020/09/iita1.jpg"></a>

Ceci est le **notebook de Deep Learning pour la vision par ordinateur conçu dans le cadre de l'EEIA 2022.** Il est basé majoritairement sur **YOLOv5🚀** développé par **Ultralytics**. 

**YOLOv5🚀** est librement disponible pour la redistribution sous la licence [GPL-3.0 license](https://choosealicense.com/licenses/gpl-3.0/). 

Pour plus d'information à propos des modèles **YOLOv5🚀** et des données utilisés dans ce notebook, nous vous invitons à consulter les pages de **Ultralytics** https://github.com/ultralytics/yolov5 et **Iwaria** https://iwaria.com/fr/accueil/  Merci !

# Installation des packages et dépendances

1. Activer votre environnement virtuel et vérifier la présence dans le répertoire courant - celui de ce notebook - du fichier **requirements.txt**
2. Puis procéder à l'installation des packages indispensables à l'éxecution de ce notebook comme suit : **pip install -r requirements.txt**

# Chargement des packages

In [ ]:
import torch
from torch import hub
from PIL import Image
import utils
display = utils.notebook_init()  # checks

# Téléchargement de modèles YOLOv5 *on the shelf* via hubconf

In [ ]:
model_yolov5s = hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
model_yolov5s.eval()

In [ ]:
import yaml

with open('./data/coco.yaml', 'r') as file:
    coco_dataset = yaml.safe_load(file)

coco_dataset['names']

# 1. Inférence ou Prédiction

## a) cas facile

In [ ]:
img_easy = Image.open("./test_cases/kids_beach.jpeg")
img_easy

In [ ]:
output_easy_img = model_yolov5s(img_easy)
output_easy_img.print()
output_easy_img.save("inference_on_test_cases")
easy_img_pediction = Image.open("./inference_on_test_cases/kids_beach.jpg")
easy_img_pediction

## b) cas moyen

In [ ]:
img_moderate = Image.open("./test_cases/ecureuils_Benin.jpg")
img_moderate

In [ ]:
output_moderate_img = model_yolov5s(img_moderate)
output_moderate_img.print()
output_moderate_img.save("inference_on_test_cases")
moderate_img_pediction = Image.open("./inference_on_test_cases/ecureuils_Benin.jpg")
moderate_img_pediction

## c) cas difficile

In [ ]:
img_hard = Image.open("./test_cases/market.jpeg")
img_hard

In [ ]:
output_hard_img = model_yolov5s(img_hard)
output_hard_img.print()
output_hard_img.save("inference_on_test_cases")
hard_img_pediction = Image.open("./inference_on_test_cases/market.jpg")
hard_img_pediction

# Modification du score de confiance de prédiction et ses conséquences 

In [ ]:
help(model_yolov5s)

In [ ]:
model_yolov5s.conf = 0.3

output_moderate_img = model_yolov5s(img_moderate)
output_moderate_img.print()
output_moderate_img.save("inference_conf_change")
moderate_img_pediction = Image.open("./inference_conf_change/ecureuils_Benin.jpg")
moderate_img_pediction

# Chargement et inférence avec le modèle YOLOv5-xlarge-P6 : v5s vs v5x6

In [ ]:
model_yolov5x6 = hub.load('ultralytics/yolov5', 'yolov5x6', pretrained=True)
model_yolov5x6.eval();

In [ ]:
img_doubt = Image.open("./test_cases/trotteur.jpeg")
img_doubt

In [ ]:
model_yolov5s.conf = 0.25 # score par défaut, mais ayant été modifié plus haut, je l'ai remis à sa valeur initiale
output_doubt_img = model_yolov5s(img_doubt)
output_doubt_img.print()
output_doubt_img.save("inference_on_test_cases")
doubt_img_pediction = Image.open("./inference_on_test_cases/trotteur.jpg")
doubt_img_pediction

In [ ]:
output_doubt_img = model_yolov5x6(img_doubt)
output_doubt_img.print()
output_doubt_img.save("inference_on_test_cases")
doubt_img_pediction = Image.open("./inference_on_test_cases/trotteur.jpg")
doubt_img_pediction

# Des CNNs (YOLOv5) vers les *Vision Transformers* (DETR avec ResNet) 

Lien vers le papier/github de Facebook (devenu Meta): https://github.com/facebookresearch/detr

**NB: Pensez à redémarrer le noyau avant d'exécuter cette partie du notebook**

In [ ]:
import math

from PIL import Image
import requests
import matplotlib.pyplot as plt
#%config InlineBackend.figure_format = 'retina'

import ipywidgets as widgets
from IPython.display import display, clear_output

import torch
from torch import nn
from torchvision.models import resnet50
import torchvision.transforms as T
torch.set_grad_enabled(False);

In [ ]:
img_girl_cars = Image.open("./test_cases/girl_and_cars.jpeg")
img_girl_cars

In [ ]:
model_detr_resnet50 = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
model_detr_resnet50.eval();

In [ ]:
# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize(800),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [ ]:
def plot_results(pil_img, prob, boxes):
    plt.figure(figsize=(16,10))
    plt.imshow(pil_img)
    ax = plt.gca()
    colors = COLORS * 100
    for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
        ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                   fill=False, color=c, linewidth=3))
        cl = p.argmax()
        text = f'{CLASSES[cl]}: {p[cl]:0.2f}'
        ax.text(xmin, ymin, text, fontsize=15,
                bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    plt.show()

In [ ]:
# mean-std normalize the input image (batch-size: 1)
img_girl_cars_t = transform(img_girl_cars).unsqueeze(0)

# propagate through the model
outputs = model_detr_resnet50(img_girl_cars_t)

# keep only predictions with 0.7+ confidence
probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
keep = probas.max(-1).values > 0.9

# convert boxes from [0; 1] to image scales
bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], img_girl_cars.size)

In [ ]:
# COCO classes
CLASSES = [
    'N/A', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A',
    'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse',
    'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack',
    'umbrella', 'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis',
    'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass',
    'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich',
    'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake',
    'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table', 'N/A',
    'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A',
    'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier',
    'toothbrush'
]

# colors for visualization
COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]

In [ ]:
plot_results(img_girl_cars, probas[keep], bboxes_scaled)